# AMSC698K Homework 6
##### Elijah Kin & Noorain Noorani

In [1]:
import math
import numpy as np
import qiskit
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
qiskit.__version__

'1.2.4'

### 1a - (60pts) Create a program for Shor's algorithm, i.e. for factorizing an integer N, including a quantum circuit to find the order r of a modulo $N (a^r := 1 mod N)$. 

First, create functions that perform the required processing steps (you may adapt the corresponding functions presented in class):

1) Create helper functions:

    (a) to check whether integer N is a perfect power ($N=p^q)$;

    (b) to check whether N and the base a (with $1<a<N$) are coprime;

    (c) to extract the exponent r and the factors of N.

In [2]:
def is_perfect_power(n):
    """
    Check if integer n is a perfect power (n = p^q with q > 1).
    
    Returns:
        (True, p, q) if n is a perfect power,
        (False, None, None) otherwise.
    """
    # Loop over possible exponents q starting at 2.
    # The upper limit for q is taken from log2(n) since 2 is the smallest possible base.
    max_exponent = int(math.log(n, 2)) + 1
    for q in range(2, max_exponent + 1):
        p = int(round(n ** (1.0 / q)))
        # Check if p^q exactly equals n
        if p ** q == n:
            return True, p, q
    return False, None, None

In [3]:
def are_coprime(n, a):
    """
    Check if integer n and base a (with 1 < a < n) are coprime.
    
    Returns:
        True if gcd(n, a) == 1, False otherwise.
    """
    if not (1 < a < n):
        raise ValueError("Base a must satisfy 1 < a < n")
    return math.gcd(n, a) == 1

In [4]:
def order_and_factors(n, a):
    """
    Compute the order r of a modulo n (i.e., the smallest positive integer r 
    such that a^r ≡ 1 mod n). Then, if r is even, attempt to extract non-trivial 
    factors of n by computing:
        factor1 = gcd(a^(r/2) - 1, n)
        factor2 = gcd(a^(r/2) + 1, n)
    
    Returns:
        r: the order of a modulo n,
        factors: a tuple (factor1, factor2) if r is even, otherwise None.
    """
    # Find the order r (smallest r such that a^r mod n == 1)
    r = None
    for i in range(1, n):
        if pow(a, i, n) == 1:
            r = i
            break

    if r is None:
        raise ValueError("No order found; check inputs for coprimality.")
    
    factors = None
    # Only proceed with factor extraction if r is even.
    if r % 2 == 0:
        # Compute a^(r/2) mod n
        x = pow(a, r // 2, n)
        # Calculate possible factors using the gcd method.
        factor1 = math.gcd(x - 1, n)
        factor2 = math.gcd(x + 1, n)
        factors = (factor1, factor2)
    
    return r, factors

In [5]:
N = 16  # Try a perfect power: 16 = 2^4
print("Checking if N =", N, "is a perfect power:")
is_pp, base, exponent = is_perfect_power(N)
if is_pp:
    print(f"  Yes, {N} = {base}^{exponent}")
else:
    print("  No, it is not a perfect power.")

# Example for coprimality check:
n = 15
a = 7
print(f"\nChecking if {n} and {a} are coprime:")
print("  Coprime?" , are_coprime(n, a))

# Example for order and factor extraction:
# Here we use n = 15 and a = 7. Note that 7 and 15 are coprime.
print(f"\nExtracting order and factors for n = {n} with base a = {a}:")
r, factors = order_and_factors(n, a)
print("  Order r =", r)
if factors:
    print("  Extracted factors:", factors)
else:
    print("  Order is not even; no factors extracted via this method.")

Checking if N = 16 is a perfect power:
  Yes, 16 = 4^2

Checking if 15 and 7 are coprime:
  Coprime? True

Extracting order and factors for n = 15 with base a = 7:
  Order r = 4
  Extracted factors: (3, 5)


2) Create quantum circuits:

- to perform QFT and inverse QFT on N qubits;
- to perform the double-controlled modular addition of a in the Fourier space and its inverse;
- to perform controlled modular multiplication by a 

3) Create a program that performs the factorization:

- get a positive odd integer N (user input) and check whether N is a perfect power (function 1a);
- get base a and check whether a and N are coprime (function 1b);
with n as the number of bits in N, create 3 quantum registers: a n qubit register (initialized to 1) and a n+2 auxiliary qubit register for performing modular multiplication, and a 2n qubit register to perform the inverse QFT (initialized to uniform superposition);
- apply the modular multiplication gate of power 2k controlled by qubit k (for each k=0,2n) in the 2n qubit register;
- apply the inverse QFT on the 2n qubit register and read it out (measure all 2n qubits);
- extract the exponent r and the factors of N from the measured data (function 1c)

### 1b - (40pts) Test your program on N=15,35,55, then submit the circuit first to "ionq_simulator" without and with noise model, check the results and then run it on an IonQ QPU.

### 1c - (extra credit 30pts) Finally, choose a number 156 < N < 256, which is the product of 2 primes, and factorize N using the IonQ Forte-1 QPU.